In [128]:
import numpy as np
import pandas as pd

df0 = pd.read_csv('0_data.csv')
df1 = pd.read_csv('1_data.csv')
df2 = pd.read_csv('2_data.csv')

C:\Users\brand\AppData\Local\Temp\ipykernel_11352\135345739.py:6: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('2_data.csv')


In [129]:
# TODO: add transmission, engine something, vin -> country of manufacter, body type
df0 = df0[['vin', 'year', 'make_name', 'model_name', 'city', 'mileage', 'fuel_type', 'transmission', 'engine_cylinders','wheel_system', 'price']]
df0.columns = ['vin', 'year', 'make', 'model', 'city', 'mileage', 'fuel','transmission', 'engine','drive', 'price']

df1 = df1[['VIN', 'year', 'manufacturer', 'model', 'region', 'odometer', 'fuel','transmission','cylinders','drive', 'price']]
df1.columns = ['vin', 'year', 'make', 'model', 'city', 'mileage', 'fuel','transmission', 'engine','drive', 'price']

df2 = df2[['vin', 'year', 'make', 'model', 'city', 'miles', 'fuel_type', 'transmission', 'engine_size','drivetrain', 'price']]
df2.columns = ['vin', 'year', 'make', 'model', 'city', 'mileage', 'fuel','transmission', 'engine','drive', 'price']

frames=[df0,df1,df2]
df = pd.concat(frames)
display(df)

,vin,year,make,model,city,mileage,fuel,transmission,engine,drive,price
0,ZACNJABB5KPJ92081,2019.0,Jeep,Renegade,Bayamon,7.0,Gasoline,A,I4,FWD,23141.0
1,SALCJ2FX1LH858117,2020.0,Land Rover,Discovery Sport,San Juan,8.0,Gasoline,A,I4,AWD,46500.0
2,JF1VA2M67G9829723,2016.0,Subaru,WRX STI,Guaynabo,NaN,Gasoline,M,H4,AWD,46995.0
3,SALRR2RV0L2433391,2020.0,Land Rover,Discovery,San Juan,11.0,Gasoline,A,V6,AWD,67430.0
4,SALCJ2FXXLH862327,2020.0,Land Rover,Discovery Sport,San Juan,7.0,Gasoline,A,I4,AWD,48880.0
...,...,...,...,...,...,...,...,...,...,...,...
99995,WA1ANAFY0J2138713,2018.0,Audi,Q5,Selma,29085.0,Premium Unleaded,Automatic,2.0,4WD,31395.0
99996,JF1VA1A60K9818070,2019.0,Subaru,WRX,Charlotte,30003.0,Premium Unleaded,Manual,2.0,4WD,29990.0
99997,WBXYH9C03L5P79755,2020.0,BMW,X2,Murrieta,8825.0,Premium Unleaded,Automatic,2.0,FWD,35991.0
99998,JTJYARBZ7H2082838,2017.0,Lexus,NX,Buford,56955.0,Premium Unleaded,Automatic,2.0,FWD,27960.0


In [130]:
backup = df.copy()

# Preprocess

In [131]:
# cleaning

#dropping null values
df=df.dropna()

#dropping any cars less than 100 dollars (all of the dropped ones actually have a price of 0)
df=df.loc[df['price']>100]

# standardizing the make feature
def normal_string(s):
    return s.lower().replace(' ', '').replace('-', '').replace('_', '')
 
df['model'] = df['model'].apply(normal_string)
df['make'] = df['make'].apply(normal_string)
df['city'] = df['city'].apply(normal_string)
df['drive'] = df['drive'].apply(normal_string)


# standardizing the fuel feature
gasoline = {'gas', 
            'E85 / Unleaded',
       'Unleaded', 'Premium Unleaded',
       'Premium Unleaded; Unleaded', 
       'Unleaded; Unleaded / E85', 'Unleaded / E85',
       'E85 / Unleaded; Unleaded', 'Premium Unleaded / Unleaded',
       'E85 / Premium Unleaded; E85 / Unleaded', 
       'E85', 'E85 / Premium Unleaded', 
       'Compressed Natural Gas; Unleaded',
       'E85 / Unleaded; Unleaded / Unleaded',
       'Diesel / Premium Unleaded', 'E85 / Unleaded; Unleaded / E85',
       'Unleaded / Unleaded', 
       'Compressed Natural Gas / Unleaded', 'Diesel; Unleaded',
       'Diesel; E85 / Unleaded', 'E85 / Unleaded; Premium Unleaded',
       'Premium Unleaded; Premium Unleaded / E85', 'E85; E85 / Unleaded',
       'Unleaded / Premium Unleaded',
       'Premium Unleaded / E85',
       'M85 / Unleaded'
}

diesel = {
    'Biodiesel'
}

hybrid = {
       'Electric / Premium Unleaded', 
       'Electric / Unleaded',
       'Unleaded / Electric',
       'Electric / Hydrogen',
       'Electric / Premium Unleaded; Electric / Unleaded',
       'Electric / Premium Unleaded; Premium Unleaded',
       'Electric / E85'
}

other = {
    'Hydrogen', 
    'Premium Unleaded / Natural Gas',
    'Compressed Natural Gas / Lpg', 
    'Compressed Natural Gas', 'Propane',
    'Flex Fuel Vehicle',
}

def fuel_preprocess(x):
    if x in gasoline:
        return 'gasoline'
    if x in diesel:
        return 'diesel'
    if x in hybrid:
        return 'hybrid'
    if x in other:
        return 'other'
    return x


df['fuel'] = df['fuel'].apply(fuel_preprocess)

def find_engine_size(s):
    if type(s) == float:
        return s
    elif type(s) == str:
        n_str = ''
        for c in s:
            if c.isnumeric() or c == '.':
                n_str += c
        if len(n_str) > 0:
            return float(n_str)/2
        else:
            return(np.nan)
    else:
        print('other type')
        return(np.nan)

df['engine'] = df['engine'].apply(find_engine_size)
df = df.dropna()

# explore top x most common variables

In [132]:
x = 10

cat_list = ['make', 'model', 'city', 'fuel', 'transmission', 'drive']
cat_df = df[cat_list]

def get_top_ratio(column, x):
    top_x = sum(column.value_counts().tolist()[:x])
    total = column.value_counts().sum()
    return top_x / total

for column in cat_df:
    ratio = get_top_ratio(df[column], x) * 100
    print(f'top {x} variables of {column} make up {np.round(ratio, 2)}% our its values')

top 10 variables of make make up 65.6% our its values
top 10 variables of model make up 16.24% our its values
top 10 variables of city make up 7.55% our its values
top 10 variables of fuel make up 100.0% our its values
top 10 variables of transmission make up 100.0% our its values
top 10 variables of drive make up 100.0% our its values


# one hot encoding

In [133]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse_output = False, max_categories = x)
cat_df = enc.fit_transform(cat_df)
cat_df = pd.DataFrame(cat_df)
cat_df.columns = enc.get_feature_names_out()
cat_df.head()

df = df.drop(cat_list, axis = 1)
df = cat_df.join(df, how='outer')

In [134]:
df.head()

,make_bmw,make_chevrolet,make_dodge,make_ford,make_honda,make_jeep,make_mercedesbenz,make_nissan,make_toyota,make_infrequent_sklearn,model_challenger,model_ecosport,model_equinox,model_escape,model_f150,model_fusion,model_grandcherokee,model_ram2500pickup,model_silverado1500,model_infrequent_sklearn,city_anchorage/matsu,city_bronx,city_brooklyn,city_clintontownship,city_danbury,city_novi,city_rochesterhills,city_stamford,city_waterbury,city_infrequent_sklearn,fuel_Diesel,fuel_Gasoline,fuel_Hybrid,fuel_diesel,fuel_electric,fuel_gasoline,fuel_hybrid,fuel_other,transmission_A,transmission_Automatic,transmission_CVT,transmission_Dual Clutch,transmission_M,transmission_Manual,transmission_automatic,transmission_manual,transmission_other,drive_4wd,drive_4x2,drive_awd,drive_fwd,drive_rwd,vin,year,mileage,engine,price
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,ZACNJABB5KPJ92081,2019.0,7.0,2.0,23141.0
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1GCWGFCF3F1284719,2015.0,115879.0,4.8,20998.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,SALCJ2FX1LH858117,2020.0,8.0,2.0,46500.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,WBY7Z8C59JVB87514,2018.0,7339.0,0.6,27921.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,ML32F4FJ2JHF10325,2018.0,39798.0,1.2,11055.0


In [ ]:
almost_df = df.drop('vin', axis=1)

almost_df = almost_df.sample(frac=0.2, random_state = 2)

almost_df.memory_usage(index=True).sum()/1000000

#almost_df.to_csv('almost_df.csv')